In [1]:
import pandas as pd

import requests
import json
import pandas as pd
from itertools import chain
import time
import concurrent.futures
from requests.exceptions import HTTPError
import json
import ast
import re
import zipfile
import string

import matplotlib.pyplot as plot
import numpy as np

!pip install transformers
!pip install keyphrasetransformer
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer

#download and define embedding model
model = SentenceTransformer('allenai-specter')

  Preparing metadata (setup.py) ... done
  Created wheel for keyphrasetransformer: filename=keyphrasetransformer-0.0.2-py3-none-any.whl size=4180 sha256=4ccc4357f1904b7e7cfcb209272bd8a7dab207e86b285aa58044ec3f1e6167d2
  Stored in directory: /root/.cache/pip/wheels/a6/84/70/8a043e66f4f5dd346129170b3729f80985226b4aace0f2795b
Successfully built keyphrasetransformer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/331 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/462k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
# Connect notebook to Drive (left panel) by mounting drive

people_by = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/baylor_people_year0.csv')
people_gt = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/extra data/gtech_people.csv')
people_hp = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/extra data/hopkins_people.csv')

#people_1 = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/people.csv')

grants_by = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/baylor_grants.csv')
grants_gt = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/extra data/gtech_grants.csv')
grants_hp = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/extra data/hopkins_grants.csv')

# papers = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/papers.csv')
papers_by = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/unrolled_papers_df.csv')
papers_gt = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/extra data/gtech_papers.csv')
papers_hp = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/extra data/hopkins_papers.csv')


companies_by = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/baylor_companies.csv')
companies_gt = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/extra data/gtech_companies.csv')
companies_hp = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/extra data/hopkins_companies.csv')


patents_by = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/baylor_inventors.csv')
patents_gt = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/extra data/gtech_inventors.csv')
patents_hp = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/extra data/hopkins_inventors.csv')


sample_inventors = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/sample_inventors_with_startups_baylor.csv')
sample_inventors=sample_inventors.fillna(0)




In [3]:
people = pd.concat([pd.DataFrame(people_by), pd.DataFrame(people_gt), pd.DataFrame(people_hp)], axis=0)
people = people.reset_index(drop=True)

grants = pd.concat([pd.DataFrame(grants_by), pd.DataFrame(grants_gt), pd.DataFrame(grants_hp)], axis=0)
grants = grants.reset_index(drop=True)

papers = pd.concat([pd.DataFrame(papers_by), pd.DataFrame(papers_gt), pd.DataFrame(papers_hp)], axis=0)
papers = papers.reset_index(drop=True)

companies = pd.concat([pd.DataFrame(companies_by), pd.DataFrame(companies_gt), pd.DataFrame(companies_hp)], axis=0)
companies = companies.reset_index(drop=True)

patents = pd.concat([pd.DataFrame(patents_by), pd.DataFrame(patents_gt), pd.DataFrame(patents_hp)], axis=0)
patents = patents.reset_index(drop=True)


In [4]:
for i in range(len(companies['portal_index_id'])):
    id = companies['portal_index_id'].iloc[i]
    if id.count('[') > 0:
        if id.count(',') > 0:
            if id[2:13] != id[17:28]:
                new_index = len(companies)  # Get the index label for the new row
                companies.loc[new_index] = companies.iloc[i]  # Append a new row with the same values
                companies.at[i, 'portal_index_id'] = id[2:13]  # Modify the value using .at[]
                companies.at[new_index, 'portal_index_id'] = id[17:28]
            else:
                companies.at[i, 'portal_index_id'] = id[2:13]  # Modify the value using .at[]
        else:
            companies.at[i, 'portal_index_id'] = id[2:13]  # Modify the value using .at[]


In [5]:
def sample(data):

    df = pd.DataFrame(data)
    sample_data = []

    for id in companies['portal_index_id']:
        #print(id)
        # Filter the DataFrame for rows where 'portal_index_id' matches the id
        matches = df[df['portal_index_id'].astype(str).str.contains(str(id))]
        #print(matches['name'])

        # If matches exist, append all matching rows to sample_data
        if not matches.empty:
            sample_data.extend(matches.values.tolist())
        else:
            sample_data.append([ ])

    # Add the column names as the first row of sample_data
    sample_data = [df.columns.tolist()] + sample_data

    # Create a DataFrame from sample_data
    sample_df = pd.DataFrame(sample_data)

    # Set the column names from the first row
    sample_df.columns = sample_df.iloc[0]

    # Remove the first row (which contains column names) from sample_df
    sample_df = sample_df[1:]


    # Iterate over the DataFrame
    index_to_drop = []
    for i in range(len(sample_df) - 1):
        if sample_df.iloc[i].equals(sample_df.iloc[i + 1]):
            # Mark the index of the row to be dropped
            index_to_drop.append(i + 1)

    # Drop the marked rows
    sample_df = sample_df.drop(index_to_drop)

    sample_df = sample_df.reset_index(drop=True)

    return sample_df


In [6]:
people = people.dropna(subset=['earliest_consecutive_year'])

people_sample    = []
grants_sample    = []
papers_sample    = []
companies_sample = []
patents_sample   = []

people_sample = sample(people)
#grants_sample = sample(grants)
#papers_sample = sample(papers)
#papers_sample_new = sample(papers_new)
#companies_sample = sample(companies)
#patents_sample = sample(patents)

people_sample = people_sample.dropna(subset=['earliest_consecutive_year'])

In [7]:
people_sample

,Unnamed: 0,name,portal_index_id,institution_id,institution,past_known_id,works_count,citation_count,h_index,mean_citedness,other_names,past_known,earliest_consecutive_year
0,241.0,Charles Y. Lin,A5032502969,02pttbw34,Baylor College of Medicine,"['https://openalex.org/I181547552', 'https://o...",180,15074,43,8.481481,"['Lin Charles', 'Charles Y. Lin', 'Charles Lin']","['I181547552', 'I2801539370']",2017.0
1,99.0,Ann M. Leen,A5020956671,02pttbw34,Baylor College of Medicine,"['https://openalex.org/I181547552', 'https://o...",337,8627,50,4.176471,"['Ann Leen', 'Ann M. Leen', 'A. Leen', 'Ann Ma...","['I181547552', 'I2801539370']",2002.0
2,24.0,Joshua Cohen,A5066250865,02pttbw34,Baylor College of Medicine,"['https://openalex.org/I181547552', 'https://o...",717,4572,23,0.035714,"['J Laurence Cohen', 'Joshua Benjamin Cohen', ...","['I181547552', 'I2801539370']",2020.0
3,155.0,Robert A. Brown,A5054721642,02pttbw34,Baylor College of Medicine,"['https://openalex.org/I181547552', 'https://o...",256,4064,35,2.357143,"['Robert Andrew Brown', 'Robert A. Brown', 'Rh...","['I181547552', 'I2801539370']",1975.0
4,4772.0,Stephanie Smith,A5002585034,03nxfhe13,Baylor University Medical Center,"['https://openalex.org/I2802366037', 'https://...",130,4256,26,38.136364,"['Stephanie Jay Smith', 'Sharda Jackson Smith'...","['I2802366037', 'I4210160418']",2002.0
5,870.0,David G. Edwards,A5064093388,02pttbw34,Baylor College of Medicine,"['https://openalex.org/I181547552', 'https://o...",60,288,9,1.0,"['D.G. Edwards', 'D.G Edwards', 'David Edwards...","['I181547552', 'I2801539370']",2023.0
6,11.0,Matthew J. Ellis,A5057135409,02pttbw34,Baylor College of Medicine,"['https://openalex.org/I181547552', 'https://o...",969,61151,96,9.736111,"['Matthew James Ellis', 'M. Ellis', 'Matthew J...","['I181547552', 'I2801539370']",1993.0
7,365.0,Chris Perkins,A5022368721,02pttbw34,Baylor College of Medicine,"['https://openalex.org/I181547552', 'https://o...",131,2186,24,0.0,"['Christopher Perkins', 'C. Perkins', 'Clare P...","['I181547552', 'I2801539370']",2012.0
8,3486.0,Lei Qi,A5004306472,02pttbw34,Baylor College of Medicine,"['https://openalex.org/I181547552', 'https://o...",3,7,2,2.0,['Lei Qi'],"['I181547552', 'I2801539370']",2023.0
9,3842.0,Cory M. Smith,A5075327892,005781934,Baylor University,['https://openalex.org/I157394403'],191,2115,21,0.578947,"['Cory Smith', 'C. P. Smith', 'Charles P. Smit...",['I157394403'],1955.0


In [8]:
for i in range(len(companies)):
    if pd.isna(companies['Most Recent Valuation (MM)'].loc[i]):
        companies.loc[companies['Most Recent Valuation (MM)'].isna(), 'Most Recent Valuation (MM)'] = companies['Raised to Date ($MM)']

In [9]:
grants = grants.dropna(subset=['project_start_date'])


# **Year 0 - 10**

In [10]:

variables_0_10 = pd.DataFrame(index=people_sample['portal_index_id'],columns = ['career length','paper frequency','paper citation frequency','paper citation per paper',\
                                                                                'author position','is_corresponding',\
                                                                                'grant number', 'grant money','grant money per grant','grant series_u','grant series_r',\
                                                                                'grant series_t','grant series_p','grant duration per grant' ,  \
                                                                                'patents citations frequency' ,'patents frequency', \
                                                                                'Startup value (mm$)', 'Startup value_0/1'])

nn = variables_0_10.shape[1]

for id in people_sample['portal_index_id']:

        variables_0_10['Startup value (mm$)'].loc[id] = companies[companies['portal_index_id'].astype(str).str.contains(str(id))]['Raised to Date ($MM)'].values[0]


        ################ career length ############
        variables_0_10['career length'].loc[id] = 1


        ################ paper frequency ############
        #paper_max = (papers[papers['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).max()
        #paper_min = (papers[papers['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).min()
        #print(paper_max,paper_min)
        freq = 0
        paper_min = people_sample[people_sample['portal_index_id'].astype(str).str.contains(str(id))]['earliest_consecutive_year'].values.astype(int)
        if paper_min[0] > 2013:
            freq = papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 2013) ]['publication_year'].dropna().count() / (2023 - paper_min + 1)
        else:
            freq = papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] < paper_min[0]+10) ]['publication_year'].dropna().count() / 10
        variables_0_10['paper frequency'].loc[id] = freq


        ################ paper citation ############
        freq = 0
        if paper_min[0] > 2013:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 2013) ]['forward_citations'].sum()) / (2023 - paper_min + 1)
        else:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] < paper_min[0]+10) ]['forward_citations'].sum()) / 10
            #print(id,paper_min)
        variables_0_10['paper citation frequency'].loc[id] = freq

        ################# paper citation per paper ############
        freq = 0
        if paper_min[0] > 2013:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 2013) ]['forward_citations'].sum() ) / papers[ (papers['portal_index_id'].astype(str).str.contains(str(id)))&(papers['publication_year'] >= 2013) ]['publication_year'].dropna().count()
        else:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] < paper_min[0]+10) ]['forward_citations'].sum() ) / papers[ (papers['portal_index_id'].astype(str).str.contains(str(id)))&(papers['publication_year'] < paper_min[0]+10)  ]['publication_year'].dropna().count()
            #print(id,paper_min)
        variables_0_10['paper citation per paper'].loc[id] = freq


        ################# grant number frequency ############
        freq = 0
        if paper_min[0] > 2013:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['project_start_date'].dropna().count() ) / (2023 - paper_min + 1)
            # print(float(grants[
            #                 (grants['portal_index_id'].astype(str).str.contains(str(id))) &
            #                 (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['project_start_date'].dropna().count()  ))
            # print(freq)
        else:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['project_start_date'].dropna().count() ) / 10
            #print(id,paper_min)
        variables_0_10['grant number'].loc[id] = freq


        # if paper_min[0] > 2013:
        #     print(float(grants[
        #                   (grants['portal_index_id'].astype(str).str.contains(str(id))) &
        #                   (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['project_start_date'].dropna().count()  ))
        #     print(grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
        #                     (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['project_end_date'].str[:4].astype(int).sum())

        ################# grant money frequency ############
        freq = 0
        if paper_min[0] > 2013:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['award_amount'].sum() ) / (2023 - paper_min + 1)
        else:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['award_amount'].sum() ) / 10
            #print(id,paper_min)
        variables_0_10['grant money'].loc[id] = freq


        ################# grant money per grant ############
        freq = 0
        if paper_min[0] > 2013:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['fiscal_year'].dropna().count() )
            if num != 0:
              freq = float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['award_amount'].sum() ) / float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['fiscal_year'].dropna().count() )
        else:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['fiscal_year'].dropna().count() )
            if num != 0:
              freq = float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['award_amount'].sum() ) / float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['fiscal_year'].dropna().count() )
              #print(id,paper_min)
        variables_0_10['grant money per grant'].loc[id] = freq

        ################# grant series ############
        if paper_min[0] > 2013:
            uu = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'U')  &
                (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['award_amount'].dropna().count()
        else:
            uu = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'U')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10)  ]['award_amount'].dropna().count()
        variables_0_10['grant series_u'].loc[id] = uu


        if paper_min[0] > 2013:
            rr = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'R')  &
                (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['award_amount'].dropna().count()
        else:
            rr = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'R')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10)  ]['award_amount'].dropna().count()
        variables_0_10['grant series_r'].loc[id] = rr


        if paper_min[0] > 2013:
            tt = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'T')  &
                (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['award_amount'].dropna().count()
        else:
            tt = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'T')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10)  ]['award_amount'].dropna().count()
        variables_0_10['grant series_t'].loc[id] = tt

        if paper_min[0] > 2013:
            pp = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'P')  &
                (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['award_amount'].dropna().count()
        else:
            pp = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'P')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10)  ]['award_amount'].dropna().count()
        variables_0_10['grant series_p'].loc[id] = pp


        ################# grant duration per grant ############
        freq = 0
        if paper_min[0] > 2013:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['fiscal_year'].dropna().count() )
            if num != 0.0:
                end = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                            (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['project_end_date'].str[:4].astype(int).sum()
                start = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                              (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['project_start_date'].str[:4].astype(int).sum()
                freq = ( end - start )/ num
        else:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['fiscal_year'].dropna().count() )
            #print(num)
            if num != 0.0:
                end = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                            (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['project_end_date'].str[:4].astype(int).sum()
                start = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                              (grants['project_start_date'].str[:4].astype(int)< paper_min[0]+10) ]['project_start_date'].str[:4].astype(int).sum()
                freq = ( end - start )/ num
        variables_0_10['grant duration per grant'].loc[id] = freq




        ################# author position ############
        if paper_min[0] > 2013:
            first = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'first')  &
                (papers['publication_year'] >= 2013) ]
            middle = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'middle') &
                (papers['publication_year'] >= 2013) ]
            last = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'last')   &
                (papers['publication_year'] >= 2013) ]
            aupo = 5 * first['publication_year'].dropna().count() + 3 * middle['publication_year'].dropna().count() + 1 * last['publication_year'].dropna().count() / (2023 - paper_min + 1)
        else:
            first = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'first')  &
                (papers['publication_year']  < paper_min[0]+10) ]
            middle = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'middle') &
                (papers['publication_year']  < paper_min[0]+10) ]
            last = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'last')   &
                (papers['publication_year']  < paper_min[0]+10) ]
            aupo = 5 * first['publication_year'].dropna().count() + 3 * middle['publication_year'].dropna().count() + 1 * last['publication_year'].dropna().count() /10
        variables_0_10['author position'].loc[id] = aupo



        ################# is corresponding ############

        if paper_min[0] > 2013:
            croT = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['is_corresponding'] == True)&
                (papers['publication_year'] >= 2013) ]
            isco =  croT['publication_year'].dropna().count()
            variables_0_10['is_corresponding'].loc[id] = isco / (2023 - paper_min + 1)
        else:
            croT = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['is_corresponding'] == True)&
                (papers['publication_year']< paper_min[0]+10)]
            isco =  croT['publication_year'].dropna().count()
            variables_0_10['is_corresponding'].loc[id] = isco / 10


        ################ patents frequency ############
        freq = 0
        if paper_min[0] > 2013:
            freq = patents[
                        (patents['portal_index_id'].astype(str).str.contains(str(id))) &
                        (patents['publication_year'] >= 2013) ]['publication_year'].dropna().count() / (2023 - paper_min + 1)
        else:
            freq = patents[
                        (patents['portal_index_id'].astype(str).str.contains(str(id))) &
                        (patents['publication_year'] < paper_min[0]+10) ]['publication_year'].dropna().count() / 10
            #print(id,paper_min)
        variables_0_10['patents frequency'].loc[id] = freq

        # ################ patents citations ############

        if paper_min[0] > 2013:
            patcita = float(patents.loc[
                (patents['portal_index_id'].astype(str).str.contains(str(id)))&
                (papers['publication_year'] >= 2013)]['npl_cite_count'].sum()) / (2023 - paper_min + 1)
            variables_0_10['patents citations frequency'].loc[id] = patcita
        else:
            patcita = float(patents.loc[
                (patents['portal_index_id'].astype(str).str.contains(str(id)))&
                (papers['publication_year']  < paper_min[0]+10)]['npl_cite_count'].sum()) / 10
            variables_0_10['patents citations frequency'].loc[id] = patcita


#sample_inventors_new = sample_inventors
#sample_inventors_new.index = people_sample['portal_index_id']

#variables_0_10['Startup value (mm$)'] = sample_inventors_new['Startup value (mm$)']

#variables_0_10['Total score'] = (variables_0_10.iloc[:,0:nn]).sum(axis=1)/nn #(scores.iloc[:,0]+scores.iloc[:,1]+scores.iloc[:,2]+scores.iloc[:,3]+scores.iloc[:,4]+5*scores.iloc[:,5])/nn #

# for col in ['Startup value (mm$)']:
#     min_val = variables_0_10[col].min()
#     max_val = variables_0_10[col].max()
#     variables_0_10[col] = (variables_0_10[col] - min_val) / (max_val - min_val)


variables_0_10['Startup value (mm$)'] = variables_0_10['Startup value (mm$)'].str.replace(',', '').str.replace('$', '').str.strip().fillna(0).astype(float)

variables_0_10['Startup value_0/1'] = (variables_0_10['Startup value (mm$)'] > 5).astype(int)


variables_0_10 = variables_0_10.astype(float)

variables_0_10 = variables_0_10.round(3)

variables_0_10=variables_0_10.fillna(0)


<ipython-input-10-9162decd6636>:52: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-10-9162decd6636>:56: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-10-9162decd6636>:56: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-10-9162decd6636>:56: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-10-9162decd6636>:56: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-10-9162decd6636>:56: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-10-9162decd6636>:56: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-10-9162decd6636>:56: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-10-9162decd6636>:56: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<

# **Year 10 - 20**

In [ ]:

variables_10_20 = pd.DataFrame(index=people_sample['portal_index_id'],columns = ['career length','paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
                                                                                'author position','author position_delta','is_corresponding','is_corresponding_delta',\
                                                                                'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
                                                                                'grant series_t','grant series_p','grant duration per grant' ,  \
                                                                                'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta', \
                                                                                'Startup value (mm$)'])

nn = variables_10_20.shape[1]

for id in people_sample['portal_index_id']:

        variables_10_20['Startup value (mm$)'].loc[id] = companies[companies['portal_index_id'].astype(str).str.contains(str(id))]['Raised to Date ($MM)'].values[0]


        ################ paper frequency ############
        #paper_max = (papers[papers['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).max()
        #paper_min = (papers[papers['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).min()
        #print(paper_max,paper_min)
        freq = 0
        paper_min = people_sample[people_sample['portal_index_id'].astype(str).str.contains(str(id))]['earliest_consecutive_year'].values.astype(int)
        if paper_min[0] > 2003:
            freq = papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 2003) & (papers['publication_year'] < 2013)]['publication_year'].dropna().count() / (2013 - paper_min + 1)
        else:
            freq = papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] < paper_min[0]+20) & (papers['publication_year'] >= paper_min[0]+10) ]['publication_year'].dropna().count() / 10
        variables_10_20['paper frequency'].loc[id] = freq

        ################ career length ############
        if paper_min[0] > 2013:
          variables_10_20['career length'].loc[id] = 0
        else:
          variables_10_20['career length'].loc[id] = 1


        ################ paper citation ############
        freq = 0
        if paper_min[0] > 2003:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 2003) & (papers['publication_year'] < 2013)]['forward_citations'].sum()) / (2013 - paper_min + 1)
        else:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] < paper_min[0]+20) & (papers['publication_year'] >= paper_min[0]+10) ]['forward_citations'].sum()) / 10
            #print(id,paper_min)
        variables_10_20['paper citation frequency'].loc[id] = freq

        ################# paper citation per paper ############
        freq = 0
        if paper_min[0] > 2003:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 2003) &  (papers['publication_year'] < 2013)]['forward_citations'].sum() ) / papers[ (papers['portal_index_id'].astype(str).str.contains(str(id)))&
                        (papers['publication_year'] >= 2003) &  (papers['publication_year'] < 2013) ]['publication_year'].dropna().count()
        else:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] < paper_min[0]+20) & (papers['publication_year'] >= paper_min[0]+10) ]['forward_citations'].sum() ) / papers[ (papers['portal_index_id'].astype(str).str.contains(str(id)))&
                        (papers['publication_year'] < paper_min[0]+20) & (papers['publication_year'] >= paper_min[0]+10) ]['publication_year'].dropna().count()
            #print(id,paper_min)
        variables_10_20['paper citation per paper'].loc[id] = freq


        ################# grant number frequency ############
        freq = 0
        if paper_min[0] > 2003:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) >= 2003) & (grants['project_start_date'].str[:4].astype(int) < 2013) ]['project_start_date'].dropna().count() ) / (2013 - paper_min + 1)
        else:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+20) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+10) ]['project_start_date'].dropna().count() ) / 10
            #print(id,paper_min)
        variables_10_20['grant number'].loc[id] = freq


        ################# grant money frequency ############
        freq = 0
        if paper_min[0] > 2003:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) >= 2003) & (grants['project_start_date'].str[:4].astype(int) < 2013) ]['award_amount'].sum() ) / (2013 - paper_min + 1)
        else:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+20) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+10) ]['award_amount'].sum() ) / 10
            #print(id,paper_min)
        variables_10_20['grant money'].loc[id] = freq


        ################# grant money per grant ############
        freq = 0
        if paper_min[0] > 2003:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) >= 2003) & (grants['project_start_date'].str[:4].astype(int) < 2013)]['fiscal_year'].dropna().count() )
            if num != 0:
              freq = float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) >= 2003) & (grants['project_start_date'].str[:4].astype(int) < 2013)  ]['award_amount'].sum() ) / float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) >= 2003) & (grants['project_start_date'].str[:4].astype(int) < 2013)  ]['fiscal_year'].dropna().count() )
        else:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+20) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+10) ]['fiscal_year'].dropna().count() )
            if num != 0:
              freq = float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+20) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+10) ]['award_amount'].sum() ) / float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+20) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+10) ]['fiscal_year'].dropna().count() )
              #print(id,paper_min)
        variables_10_20['grant money per grant'].loc[id] = freq

        ################# grant series ############
        if paper_min[0] > 2003:
            uu = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'U')  &
                (grants['project_start_date'].str[:4].astype(int) >= 2003) & (grants['project_start_date'].str[:4].astype(int) < 2013) ]['award_amount'].dropna().count()
        else:
            uu = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'U')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+20) &  (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+10) ]['award_amount'].dropna().count()
        variables_10_20['grant series_u'].loc[id] = uu


        if paper_min[0] > 2003:
            rr = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'R')  &
                (grants['project_start_date'].str[:4].astype(int) >= 2003) & (grants['project_start_date'].str[:4].astype(int) < 2013) ]['award_amount'].dropna().count()
        else:
            rr = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'R')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+20) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+10) ]['award_amount'].dropna().count()
        variables_10_20['grant series_r'].loc[id] = rr


        if paper_min[0] > 2003:
            tt = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'T')  &
                (grants['project_start_date'].str[:4].astype(int) >= 2003) & (grants['project_start_date'].str[:4].astype(int) < 2013) ]['award_amount'].dropna().count()
        else:
            tt = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'T')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+20) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+10)  ]['award_amount'].dropna().count()
        variables_10_20['grant series_t'].loc[id] = tt

        if paper_min[0] > 2003:
            pp = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'P')  &
                (grants['project_start_date'].str[:4].astype(int) >= 2003) & (grants['project_start_date'].str[:4].astype(int) < 2013) ]['award_amount'].dropna().count()
        else:
            pp = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'P')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+20) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+10)  ]['award_amount'].dropna().count()
        variables_10_20['grant series_p'].loc[id] = pp


        ################# grant duration per grant ############
        freq = 0
        if paper_min[0] > 2003:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) >= 2003) & (grants['project_start_date'].str[:4].astype(int) < 2013) ]['fiscal_year'].dropna().count() )
            if num != 0.0:
                end = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                            (grants['project_start_date'].str[:4].astype(int) >= 2003) & (grants['project_start_date'].str[:4].astype(int) < 2013) ]['project_end_date'].str[:4].astype(int).sum()
                start = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                              (grants['project_start_date'].str[:4].astype(int) >= 2003) & (grants['project_start_date'].str[:4].astype(int) < 2013) ]['project_start_date'].str[:4].astype(int).sum()
                freq = ( end - start )/ num
        else:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+20) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+10) ]['fiscal_year'].dropna().count() )
            #print(num)
            if num != 0.0:
                end = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                            (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+20) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+10) ]['project_end_date'].str[:4].astype(int).sum()
                start = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                              (grants['project_start_date'].str[:4].astype(int)< paper_min[0]+20) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+10) ]['project_start_date'].str[:4].astype(int).sum()
                freq = ( end - start )/ num
        variables_10_20['grant duration per grant'].loc[id] = freq




        ################# author position ############
        if paper_min[0] > 2003:
            first = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'first')  &
                (papers['publication_year'] >= 2003) & (papers['publication_year'] < 2013) ]
            middle = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'middle') &
                (papers['publication_year'] >= 2003) & (papers['publication_year'] < 2013) ]
            last = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'last')   &
                (papers['publication_year'] >= 2003) & (papers['publication_year'] < 2013) ]
            aupo = 5 * first['publication_year'].dropna().count() + 3 * middle['publication_year'].dropna().count() + 1 * last['publication_year'].dropna().count() / (2013 - paper_min + 1)
        else:
            first = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'first')  &
                (papers['publication_year']  < paper_min[0]+20) & (papers['publication_year']  >= paper_min[0]+10) ]
            middle = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'middle') &
                (papers['publication_year']  < paper_min[0]+20) & (papers['publication_year']  >= paper_min[0]+10) ]
            last = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'last')   &
                (papers['publication_year']  < paper_min[0]+20) & (papers['publication_year']  >= paper_min[0]+10) ]
            aupo = 5 * first['publication_year'].dropna().count() + 3 * middle['publication_year'].dropna().count() + 1 * last['publication_year'].dropna().count() /10
        variables_10_20['author position'].loc[id] = aupo



        ################# is corresponding ############

        if paper_min[0] > 2003:
            croT = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['is_corresponding'] == True)&
                (papers['publication_year'] >= 2003) & (papers['publication_year'] < 2013) ]
            isco =  croT['publication_year'].dropna().count()
            variables_10_20['is_corresponding'].loc[id] = isco / (2013 - paper_min + 1)
        else:
            croT = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['is_corresponding'] == True)&
                (papers['publication_year']< paper_min[0]+20) & (papers['publication_year'] >= paper_min[0]+10)]
            isco =  croT['publication_year'].dropna().count()
            variables_10_20['is_corresponding'].loc[id] = isco / 10


        ################ patents frequency ############
        freq = 0
        if paper_min[0] > 2003:
            freq = patents[
                        (patents['portal_index_id'].astype(str).str.contains(str(id))) &
                        (patents['publication_year'] >= 2003) & (patents['publication_year'] < 2013) ]['publication_year'].dropna().count() / (2013 - paper_min + 1)
        else:
            freq = patents[
                        (patents['portal_index_id'].astype(str).str.contains(str(id))) &
                        (patents['publication_year'] < paper_min[0]+20) & (patents['publication_year'] >= paper_min[0]+10) ]['publication_year'].dropna().count() / 10
            #print(id,paper_min)
        variables_10_20['patents frequency'].loc[id] = freq

        # ################ patents citations ############

        if paper_min[0] > 2003:
            patcita = float(patents.loc[
                (patents['portal_index_id'].astype(str).str.contains(str(id)))&
                (papers['publication_year'] >= 2003) & (papers['publication_year'] < 2013)]['npl_cite_count'].sum()) / (2013 - paper_min + 1)
            variables_10_20['patents citations frequency'].loc[id] = patcita
        else:
            patcita = float(patents.loc[
                (patents['portal_index_id'].astype(str).str.contains(str(id)))&
                (papers['publication_year']  < paper_min[0]+20) &  (papers['publication_year']  >= paper_min[0]+10) ]['npl_cite_count'].sum()) / 10
            variables_10_20['patents citations frequency'].loc[id] = patcita


        ################# delta ############
        variables_10_20['paper frequency_delta'] = variables_10_20['paper frequency'] - variables_0_10['paper frequency']
        variables_10_20['paper citation frequency_delta'] = variables_10_20['paper citation frequency'] - variables_0_10['paper citation frequency']
        variables_10_20['author position_delta'] = variables_10_20['author position'] - variables_0_10['author position']
        variables_10_20['is_corresponding_delta'] = variables_10_20['is_corresponding'] - variables_0_10['is_corresponding']
        variables_10_20['grant number_delta'] = variables_10_20['grant number'] - variables_0_10['grant number']
        variables_10_20['grant money_delta'] = variables_10_20['grant money'] - variables_0_10['grant money']
        variables_10_20['patents citations frequency_delta'] = variables_10_20['patents citations frequency'] - variables_0_10['patents citations frequency']
        variables_10_20['patents frequency_delta'] = variables_10_20['patents frequency'] - variables_0_10['patents frequency']


#sample_inventors_new = sample_inventors
#sample_inventors_new.index = people_sample['portal_index_id']

#variables_10_20['Startup value (mm$)'] = sample_inventors_new['Startup value (mm$)']

#variables_10_20['Total score'] = (variables_10_20.iloc[:,0:nn]).sum(axis=1)/nn #(scores.iloc[:,0]+scores.iloc[:,1]+scores.iloc[:,2]+scores.iloc[:,3]+scores.iloc[:,4]+5*scores.iloc[:,5])/nn #

# for col in ['Startup value (mm$)']:
#     min_val = variables_10_20[col].min()
#     max_val = variables_10_20[col].max()
#     variables_10_20[col] = (variables_10_20[col] - min_val) / (max_val - min_val)


variables_10_20['Startup value (mm$)'] = variables_10_20['Startup value (mm$)'].str.replace(',', '').str.replace('$', '').str.strip().fillna(0).astype(float)


variables_10_20['Startup value_0/1'] = (variables_10_20['Startup value (mm$)'] > 5).astype(int)


variables_10_20 = variables_10_20.astype(float)

variables_10_20 = variables_10_20.round(3)

variables_10_20=variables_10_20.fillna(0)


<ipython-input-11-664c776f5ecf>:54: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-11-664c776f5ecf>:54: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-11-664c776f5ecf>:54: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-11-664c776f5ecf>:59: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-11-664c776f5ecf>:54: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-11-664c776f5ecf>:54: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-11-664c776f5ecf>:54: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-11-664c776f5ecf>:54: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<ipython-input-11-664c776f5ecf>:59: RuntimeWarning: invalid value encountered in divide
  freq = float(papers[
<

# **Year 20 - 30**

In [ ]:

variables_20_30 = pd.DataFrame(index=people_sample['portal_index_id'],columns = ['career length','paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
                                                                                'author position','author position_delta','is_corresponding','is_corresponding_delta',\
                                                                                'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
                                                                                'grant series_t','grant series_p','grant duration per grant' ,  \
                                                                                'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta', \
                                                                                'Startup value (mm$)'])

nn = variables_20_30.shape[1]

for id in people_sample['portal_index_id']:

        variables_20_30['Startup value (mm$)'].loc[id] = companies[companies['portal_index_id'].astype(str).str.contains(str(id))]['Raised to Date ($MM)'].values[0]

        ################ paper frequency ############
        #paper_max = (papers[papers['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).max()
        #paper_min = (papers[papers['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).min()
        #print(paper_max,paper_min)
        freq = 0
        paper_min = people_sample[people_sample['portal_index_id'].astype(str).str.contains(str(id))]['earliest_consecutive_year'].values.astype(int)
        if paper_min[0] > 1993:
            freq = papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 1993) & (papers['publication_year'] < 2003)]['publication_year'].dropna().count() / (2003 -  paper_min + 1)
        else:
            freq = papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] < paper_min[0]+30) & (papers['publication_year'] >= paper_min[0]+20) ]['publication_year'].dropna().count() / 10
        variables_20_30['paper frequency'].loc[id] = freq



        ################ career length ############
        if paper_min[0] > 2003:
          variables_20_30['career length'].loc[id] = 0
        else:
          variables_20_30['career length'].loc[id] = 1

        ################ paper citation ############
        freq = 0
        if paper_min[0] > 1993:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 1993) & (papers['publication_year'] < 2003)]['forward_citations'].sum()) / (2003 -  paper_min + 1)
        else:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] < paper_min[0]+30) & (papers['publication_year'] >= paper_min[0]+20) ]['forward_citations'].sum()) / 10
            #print(id,paper_min)
        variables_20_30['paper citation frequency'].loc[id] = freq

        ################# paper citation per paper ############
        freq = 0
        if paper_min[0] > 1993:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 1993) &  (papers['publication_year'] < 2003)]['forward_citations'].sum() ) / papers[ (papers['portal_index_id'].astype(str).str.contains(str(id)))&
                        (papers['publication_year'] >= 1993) &  (papers['publication_year'] < 2003) ]['publication_year'].dropna().count()
        else:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] < paper_min[0]+30) & (papers['publication_year'] >= paper_min[0]+20) ]['forward_citations'].sum() ) / papers[ (papers['portal_index_id'].astype(str).str.contains(str(id)))&
                        (papers['publication_year'] < paper_min[0]+30) & (papers['publication_year'] >= paper_min[0]+20) ]['publication_year'].dropna().count()
            #print(id,paper_min)
        variables_20_30['paper citation per paper'].loc[id] = freq


        ################# grant number frequency ############
        freq = 0
        if paper_min[0] > 1993:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) >= 1993) & (grants['project_start_date'].str[:4].astype(int) < 2003) ]['project_start_date'].dropna().count() ) / (2003 -  paper_min + 1)
        else:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+30) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+20) ]['project_start_date'].dropna().count() ) / 10
            #print(id,paper_min)
        variables_20_30['grant number'].loc[id] = freq


        ################# grant money frequency ############
        freq = 0
        if paper_min[0] > 1993:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) >= 1993) & (grants['project_start_date'].str[:4].astype(int) < 2003) ]['award_amount'].sum() ) / (2003 -  paper_min + 1)
        else:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+30) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+20) ]['award_amount'].sum() ) / 10
            #print(id,paper_min)
        variables_20_30['grant money'].loc[id] = freq


        ################# grant money per grant ############
        freq = 0
        if paper_min[0] > 1993:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) >= 1993) & (grants['project_start_date'].str[:4].astype(int) < 2003)]['fiscal_year'].dropna().count() )
            if num != 0:
              freq = float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) >= 1993) & (grants['project_start_date'].str[:4].astype(int) < 2003)  ]['award_amount'].sum() ) / float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) >= 1993) & (grants['project_start_date'].str[:4].astype(int) < 2003)  ]['fiscal_year'].dropna().count() )
        else:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+30) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+20) ]['fiscal_year'].dropna().count() )
            if num != 0:
              freq = float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+30) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+20) ]['award_amount'].sum() ) / float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+30) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+20) ]['fiscal_year'].dropna().count() )
              #print(id,paper_min)
        variables_20_30['grant money per grant'].loc[id] = freq

        ################# grant series ############
        if paper_min[0] > 1993:
            uu = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'U')  &
                (grants['project_start_date'].str[:4].astype(int) >= 1993) & (grants['project_start_date'].str[:4].astype(int) < 2003) ]['award_amount'].dropna().count()
        else:
            uu = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'U')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+30) &  (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+20) ]['award_amount'].dropna().count()
        variables_20_30['grant series_u'].loc[id] = uu


        if paper_min[0] > 1993:
            rr = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'R')  &
                (grants['project_start_date'].str[:4].astype(int) >= 1993) & (grants['project_start_date'].str[:4].astype(int) < 2003) ]['award_amount'].dropna().count()
        else:
            rr = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'R')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+30) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+20) ]['award_amount'].dropna().count()
        variables_20_30['grant series_r'].loc[id] = rr


        if paper_min[0] > 1993:
            tt = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'T')  &
                (grants['project_start_date'].str[:4].astype(int) >= 1993) & (grants['project_start_date'].str[:4].astype(int) < 2003) ]['award_amount'].dropna().count()
        else:
            tt = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'T')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+30) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+20)  ]['award_amount'].dropna().count()
        variables_20_30['grant series_t'].loc[id] = tt

        if paper_min[0] > 1993:
            pp = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'P')  &
                (grants['project_start_date'].str[:4].astype(int) >= 1993) & (grants['project_start_date'].str[:4].astype(int) < 2003) ]['award_amount'].dropna().count()
        else:
            pp = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'P')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+30) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+20)  ]['award_amount'].dropna().count()
        variables_20_30['grant series_p'].loc[id] = pp


        ################# grant duration per grant ############
        freq = 0
        if paper_min[0] > 1993:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) >= 1993) & (grants['project_start_date'].str[:4].astype(int) < 2003) ]['fiscal_year'].dropna().count() )
            if num != 0.0:
                end = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                            (grants['project_start_date'].str[:4].astype(int) >= 1993) & (grants['project_start_date'].str[:4].astype(int) < 2003) ]['project_end_date'].str[:4].astype(int).sum()
                start = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                              (grants['project_start_date'].str[:4].astype(int) >= 1993) & (grants['project_start_date'].str[:4].astype(int) < 2003) ]['project_start_date'].str[:4].astype(int).sum()
                freq = ( end - start )/ num
        else:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+30) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+20) ]['fiscal_year'].dropna().count() )
            #print(num)
            if num != 0.0:
                end = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                            (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+30) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+20) ]['project_end_date'].str[:4].astype(int).sum()
                start = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                              (grants['project_start_date'].str[:4].astype(int)< paper_min[0]+30) & (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+20) ]['project_start_date'].str[:4].astype(int).sum()
                freq = ( end - start )/ num
        variables_20_30['grant duration per grant'].loc[id] = freq




        ################# author position ############
        if paper_min[0] > 1993:
            first = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'first')  &
                (papers['publication_year'] >= 1993) & (papers['publication_year'] < 2003) ]
            middle = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'middle') &
                (papers['publication_year'] >= 1993) & (papers['publication_year'] < 2003) ]
            last = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'last')   &
                (papers['publication_year'] >= 1993) & (papers['publication_year'] < 2003) ]
            aupo = 5 * first['publication_year'].dropna().count() + 3 * middle['publication_year'].dropna().count() + 1 * last['publication_year'].dropna().count() / (2003 -  paper_min + 1)
        else:
            first = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'first')  &
                (papers['publication_year']  < paper_min[0]+30) & (papers['publication_year']  >= paper_min[0]+20) ]
            middle = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'middle') &
                (papers['publication_year']  < paper_min[0]+30) & (papers['publication_year']  >= paper_min[0]+20) ]
            last = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'last')   &
                (papers['publication_year']  < paper_min[0]+30) & (papers['publication_year']  >= paper_min[0]+20) ]
            aupo = 5 * first['publication_year'].dropna().count() + 3 * middle['publication_year'].dropna().count() + 1 * last['publication_year'].dropna().count() /10
        variables_20_30['author position'].loc[id] = aupo



        ################# is corresponding ############

        if paper_min[0] > 1993:
            croT = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['is_corresponding'] == True)&
                (papers['publication_year'] >= 1993) & (papers['publication_year'] < 2003) ]
            isco =  croT['publication_year'].dropna().count()
            variables_20_30['is_corresponding'].loc[id] = isco / (2003 -  paper_min + 1)
        else:
            croT = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['is_corresponding'] == True)&
                (papers['publication_year']< paper_min[0]+30) & (papers['publication_year'] >= paper_min[0]+20)]
            isco =  croT['publication_year'].dropna().count()
            variables_20_30['is_corresponding'].loc[id] = isco / 10


        ################ patents frequency ############
        freq = 0
        if paper_min[0] > 1993:
            freq = patents[
                        (patents['portal_index_id'].astype(str).str.contains(str(id))) &
                        (patents['publication_year'] >= 1993) & (patents['publication_year'] < 2003) ]['publication_year'].dropna().count() / (2003 -  paper_min + 1)
        else:
            freq = patents[
                        (patents['portal_index_id'].astype(str).str.contains(str(id))) &
                        (patents['publication_year'] < paper_min[0]+30) & (patents['publication_year'] >= paper_min[0]+20) ]['publication_year'].dropna().count() / 10
            #print(id,paper_min)
        variables_20_30['patents frequency'].loc[id] = freq

        # ################ patents citations ############

        if paper_min[0] > 1993:
            patcita = float(patents.loc[
                (patents['portal_index_id'].astype(str).str.contains(str(id)))&
                (papers['publication_year'] >= 1993) & (papers['publication_year'] < 2003)]['npl_cite_count'].sum()) / (2003 -  paper_min + 1)
            variables_20_30['patents citations frequency'].loc[id] = patcita
        else:
            patcita = float(patents.loc[
                (patents['portal_index_id'].astype(str).str.contains(str(id)))&
                (papers['publication_year']  < paper_min[0]+30) &  (papers['publication_year']  >= paper_min[0]+20) ]['npl_cite_count'].sum()) / 10
            variables_20_30['patents citations frequency'].loc[id] = patcita

        ################# delta ############
        variables_20_30['paper frequency_delta'] = variables_20_30['paper frequency'] - variables_10_20['paper frequency']
        variables_20_30['paper citation frequency_delta'] = variables_20_30['paper citation frequency'] - variables_10_20['paper citation frequency']
        variables_20_30['author position_delta'] = variables_20_30['author position'] - variables_10_20['author position']
        variables_20_30['is_corresponding_delta'] = variables_20_30['is_corresponding'] - variables_10_20['is_corresponding']
        variables_20_30['grant number_delta'] = variables_20_30['grant number'] - variables_10_20['grant number']
        variables_20_30['grant money_delta'] = variables_20_30['grant money'] - variables_10_20['grant money']
        variables_20_30['patents citations frequency_delta'] = variables_20_30['patents citations frequency'] - variables_10_20['patents citations frequency']
        variables_20_30['patents frequency_delta'] = variables_20_30['patents frequency'] - variables_10_20['patents frequency']


#sample_inventors_new = sample_inventors
#sample_inventors_new.index = people_sample['portal_index_id']

#variables_20_30['Startup value (mm$)'] = sample_inventors_new['Startup value (mm$)']

#variables_20_30['Total score'] = (variables_20_30.iloc[:,0:nn]).sum(axis=1)/nn #(scores.iloc[:,0]+scores.iloc[:,1]+scores.iloc[:,2]+scores.iloc[:,3]+scores.iloc[:,4]+5*scores.iloc[:,5])/nn #

# for col in ['Startup value (mm$)']:
#     min_val = variables_20_30[col].min()
#     max_val = variables_20_30[col].max()
#     variables_20_30[col] = (variables_20_30[col] - min_val) / (max_val - min_val)

variables_20_30['Startup value (mm$)'] = variables_20_30['Startup value (mm$)'].str.replace(',', '').str.replace('$', '').str.strip().fillna(0).astype(float)

variables_20_30['Startup value_0/1'] = (variables_20_30['Startup value (mm$)'] > 5).astype(int)

variables_20_30 = variables_20_30.astype(float)

variables_20_30 = variables_20_30.round(3)

variables_20_30=variables_20_30.fillna(0)


# **Year 30 -**

In [ ]:

variables_30 = pd.DataFrame(index=people_sample['portal_index_id'],columns = ['career length','paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
                                                                                'author position','author position_delta','is_corresponding','is_corresponding_delta',\
                                                                                'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
                                                                                'grant series_t','grant series_p','grant duration per grant' ,  \
                                                                                'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta', \
                                                                                'Startup value (mm$)'])

nn = variables_30.shape[1]

for id in people_sample['portal_index_id']:

        variables_30['Startup value (mm$)'].loc[id] = companies[companies['portal_index_id'].astype(str).str.contains(str(id))]['Raised to Date ($MM)'].values[0]

        ################ paper frequency ############
        #paper_max = (papers[papers['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).max()
        #paper_min = (papers[papers['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).min()
        #print(paper_max,paper_min)
        freq = 0
        paper_min = people_sample[people_sample['portal_index_id'].astype(str).str.contains(str(id))]['earliest_consecutive_year'].values.astype(int)
        if paper_min[0] > 1983:
            freq = papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 1983) & (papers['publication_year'] < 1993)]['publication_year'].dropna().count() / (1993 -  paper_min + 1)
        else:
            freq = papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= paper_min[0]+30) ]['publication_year'].dropna().count() / (2023 - paper_min[0]+30 + 1)
        variables_30['paper frequency'].loc[id] = freq


        ################ career length ############
        if paper_min[0] > 1993:
          variables_30['career length'].loc[id] = 0
        else:
          variables_30['career length'].loc[id] = 1

        ################ paper citation ############
        freq = 0
        if paper_min[0] > 1983:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 1983) & (papers['publication_year'] < 1993)]['forward_citations'].sum()) / (1993 -  paper_min + 1)
        else:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= paper_min[0]+30) ]['forward_citations'].sum()) / (2023 - paper_min[0]+30 + 1)
            #print(id,paper_min)
        variables_30['paper citation frequency'].loc[id] = freq

        ################# paper citation per paper ############
        freq = 0
        if paper_min[0] > 1983:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 1983) &  (papers['publication_year'] < 1993)]['forward_citations'].sum() ) / papers[ (papers['portal_index_id'].astype(str).str.contains(str(id)))&
                        (papers['publication_year'] >= 1983) &  (papers['publication_year'] < 1993) ]['publication_year'].dropna().count()
        else:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= paper_min[0]+30) ]['forward_citations'].sum() ) / papers[ (papers['portal_index_id'].astype(str).str.contains(str(id)))&
                        (papers['publication_year'] >= paper_min[0]+30) ]['publication_year'].dropna().count()
            #print(id,paper_min)
        variables_30['paper citation per paper'].loc[id] = freq


        ################# grant number frequency ############
        freq = 0
        if paper_min[0] > 1983:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) >= 1983) & (grants['project_start_date'].str[:4].astype(int) < 1993) ]['project_start_date'].dropna().count() ) / (1993 -  paper_min + 1)
        else:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+30) ]['project_start_date'].dropna().count() ) / (2023 - paper_min[0]+30 + 1)
            #print(id,paper_min)
        variables_30['grant number'].loc[id] = freq


        ################# grant money frequency ############
        freq = 0
        if paper_min[0] > 1983:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) >= 1983) & (grants['project_start_date'].str[:4].astype(int) < 1993) ]['award_amount'].sum() ) / (1993 -  paper_min + 1)
        else:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+30) ]['award_amount'].sum() ) / (2023 - paper_min[0]+30 + 1)
            #print(id,paper_min)
        variables_30['grant money'].loc[id] = freq


        ################# grant money per grant ############
        freq = 0
        if paper_min[0] > 1983:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) >= 1983) & (grants['project_start_date'].str[:4].astype(int) < 1993)]['fiscal_year'].dropna().count() )
            if num != 0:
              freq = float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) >= 1983) & (grants['project_start_date'].str[:4].astype(int) < 1993)  ]['award_amount'].sum() ) / float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) >= 1983) & (grants['project_start_date'].str[:4].astype(int) < 1993)  ]['fiscal_year'].dropna().count() )
        else:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+30) ]['fiscal_year'].dropna().count() )
            if num != 0:
              freq = float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+30) ]['award_amount'].sum() ) / float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+30) ]['fiscal_year'].dropna().count() )
              #print(id,paper_min)
        variables_30['grant money per grant'].loc[id] = freq

        ################# grant series ############
        if paper_min[0] > 1983:
            uu = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'U')  &
                (grants['project_start_date'].str[:4].astype(int) >= 1983) & (grants['project_start_date'].str[:4].astype(int) < 1993) ]['award_amount'].dropna().count()
        else:
            uu = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'U')  &
                (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+30) ]['award_amount'].dropna().count()
        variables_30['grant series_u'].loc[id] = uu


        if paper_min[0] > 1983:
            rr = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'R')  &
                (grants['project_start_date'].str[:4].astype(int) >= 1983) & (grants['project_start_date'].str[:4].astype(int) < 1993) ]['award_amount'].dropna().count()
        else:
            rr = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'R')  &
                (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+30) ]['award_amount'].dropna().count()
        variables_30['grant series_r'].loc[id] = rr


        if paper_min[0] > 1983:
            tt = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'T')  &
                (grants['project_start_date'].str[:4].astype(int) >= 1983) & (grants['project_start_date'].str[:4].astype(int) < 1993) ]['award_amount'].dropna().count()
        else:
            tt = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'T')  &
                (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+30)  ]['award_amount'].dropna().count()
        variables_30['grant series_t'].loc[id] = tt

        if paper_min[0] > 1983:
            pp = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'P')  &
                (grants['project_start_date'].str[:4].astype(int) >= 1983) & (grants['project_start_date'].str[:4].astype(int) < 1993) ]['award_amount'].dropna().count()
        else:
            pp = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'P')  &
                (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+30)  ]['award_amount'].dropna().count()
        variables_30['grant series_p'].loc[id] = pp


        ################# grant duration per grant ############
        freq = 0
        if paper_min[0] > 1983:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) >= 1983) & (grants['project_start_date'].str[:4].astype(int) < 1993) ]['fiscal_year'].dropna().count() )
            if num != 0.0:
                end = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                            (grants['project_start_date'].str[:4].astype(int) >= 1983) & (grants['project_start_date'].str[:4].astype(int) < 1993) ]['project_end_date'].str[:4].astype(int).sum()
                start = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                              (grants['project_start_date'].str[:4].astype(int) >= 1983) & (grants['project_start_date'].str[:4].astype(int) < 1993) ]['project_start_date'].str[:4].astype(int).sum()
                freq = ( end - start )/ num
        else:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+30) ]['fiscal_year'].dropna().count() )
            #print(num)
            if num != 0.0:
                end = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                             (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+30) ]['project_end_date'].str[:4].astype(int).sum()
                start = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                               (grants['project_start_date'].str[:4].astype(int) >= paper_min[0]+30) ]['project_start_date'].str[:4].astype(int).sum()
                freq = ( end - start )/ num
        variables_30['grant duration per grant'].loc[id] = freq




        ################# author position ############
        if paper_min[0] > 1983:
            first = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'first')  &
                (papers['publication_year'] >= 1983) & (papers['publication_year'] < 1993) ]
            middle = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'middle') &
                (papers['publication_year'] >= 1983) & (papers['publication_year'] < 1993) ]
            last = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'last')   &
                (papers['publication_year'] >= 1983) & (papers['publication_year'] < 1993) ]
            aupo = 5 * first['publication_year'].dropna().count() + 3 * middle['publication_year'].dropna().count() + 1 * last['publication_year'].dropna().count() / (1993 -  paper_min + 1)
        else:
            first = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'first')  &
                (papers['publication_year']  >= paper_min[0]+30) ]
            middle = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'middle') &
                (papers['publication_year']  >= paper_min[0]+30) ]
            last = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'last')   &
                (papers['publication_year']  >= paper_min[0]+30) ]
            aupo = 5 * first['publication_year'].dropna().count() + 3 * middle['publication_year'].dropna().count() + 1 * last['publication_year'].dropna().count() / (2023 - paper_min[0]+30 + 1)
        variables_30['author position'].loc[id] = aupo



        ################# is corresponding ############

        if paper_min[0] > 1983:
            croT = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['is_corresponding'] == True)&
                (papers['publication_year'] >= 1983) & (papers['publication_year'] < 1993) ]
            isco =  croT['publication_year'].dropna().count()
            variables_30['is_corresponding'].loc[id] = isco / (1993 -  paper_min + 1)
        else:
            croT = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['is_corresponding'] == True)&
                (papers['publication_year'] >= paper_min[0]+30)]
            isco =  croT['publication_year'].dropna().count()
            variables_30['is_corresponding'].loc[id] = isco / (2023 - paper_min[0]+30 + 1)


        ################ patents frequency ############
        freq = 0
        if paper_min[0] > 1983:
            freq = patents[
                        (patents['portal_index_id'].astype(str).str.contains(str(id))) &
                        (patents['publication_year'] >= 1983) & (patents['publication_year'] < 1993) ]['publication_year'].dropna().count() / (1993 -  paper_min + 1)
        else:
            freq = patents[
                        (patents['portal_index_id'].astype(str).str.contains(str(id))) &
                        (patents['publication_year'] >= paper_min[0]+30) ]['publication_year'].dropna().count() / (2023 - paper_min[0]+30 + 1)
            #print(id,paper_min)
        variables_30['patents frequency'].loc[id] = freq

        # ################ patents citations ############

        if paper_min[0] > 1983:
            patcita = float(patents.loc[
                (patents['portal_index_id'].astype(str).str.contains(str(id)))&
                (papers['publication_year'] >= 1983) & (papers['publication_year'] < 1993)]['npl_cite_count'].sum()) / (1993 -  paper_min + 1)
            variables_30['patents citations frequency'].loc[id] = patcita
        else:
            patcita = float(patents.loc[
                (patents['portal_index_id'].astype(str).str.contains(str(id)))&
               (papers['publication_year']  >= paper_min[0]+30) ]['npl_cite_count'].sum()) / (2023 - paper_min[0]+30 + 1)
            variables_30['patents citations frequency'].loc[id] = patcita


        ################# delta ############
        variables_30['paper frequency_delta'] = variables_30['paper frequency'] - variables_20_30['paper frequency']
        variables_30['paper citation frequency_delta'] = variables_30['paper citation frequency'] - variables_20_30['paper citation frequency']
        variables_30['author position_delta'] = variables_30['author position'] - variables_20_30['author position']
        variables_30['is_corresponding_delta'] = variables_30['is_corresponding'] - variables_20_30['is_corresponding']
        variables_30['grant number_delta'] = variables_30['grant number'] - variables_20_30['grant number']
        variables_30['grant money_delta'] = variables_30['grant money'] - variables_20_30['grant money']
        variables_30['patents citations frequency_delta'] = variables_30['patents citations frequency'] - variables_20_30['patents citations frequency']
        variables_30['patents frequency_delta'] = variables_30['patents frequency'] - variables_20_30['patents frequency']



#sample_inventors_new = sample_inventors
#sample_inventors_new.index = people_sample['portal_index_id']

#variables_30['Startup value (mm$)'] = sample_inventors_new['Startup value (mm$)']

#variables_30['Total score'] = (variables_30.iloc[:,0:nn]).sum(axis=1)/nn #(scores.iloc[:,0]+scores.iloc[:,1]+scores.iloc[:,2]+scores.iloc[:,3]+scores.iloc[:,4]+5*scores.iloc[:,5])/nn #

# for col in ['Startup value (mm$)']:
#     min_val = variables_30[col].min()
#     max_val = variables_30[col].max()
#     variables_30[col] = (variables_30[col] - min_val) / (max_val - min_val)

variables_30['Startup value (mm$)'] = variables_30['Startup value (mm$)'].str.replace(',', '').str.replace('$', '').str.strip().fillna(0).astype(float)

variables_30['Startup value_0/1'] = (variables_30['Startup value (mm$)'] > 5).astype(int)

variables_30 = variables_30.astype(float)

variables_30 = variables_30.round(3)

variables_30=variables_30.fillna(0)


# **Correlation**

In [ ]:
variables_0_10_new = variables_0_10[variables_0_10.iloc[:, 0] != 0]
variables_10_20_new = variables_10_20[variables_10_20.iloc[:, 0] != 0]
variables_20_30_new = variables_20_30[variables_20_30.iloc[:, 0] != 0]
variables_30_new = variables_30[variables_30.iloc[:, 0] != 0]

variables_0_10_new = variables_0_10_new.iloc[:,1:]
variables_10_20_new = variables_10_20_new.iloc[:,1:]
variables_20_30_new = variables_20_30_new.iloc[:,1:]
variables_30_new = variables_30_new.iloc[:,1:]

In [ ]:
# variables_0_10_new.iloc[:, -1] = (variables_0_10_new.iloc[:, -1] > 5).astype(int)
# variables_10_20_new.iloc[:, -1] = (variables_10_20_new.iloc[:, -1] > 5).astype(int)
# variables_20_30_new.iloc[:, -1] = (variables_20_30_new.iloc[:, -1] > 5).astype(int)
# variables_30_new.iloc[:, -1] = (variables_30_new.iloc[:, -1] > 5).astype(int)


In [ ]:
# Select all columns except the last one
variables_except_last = variables_0_10_new.iloc[:, :-1]

# Calculate the correlation between the selected columns and the 'Startup value (mm$)' column
pearson_corr_0_10 = variables_except_last.corrwith(variables_0_10_new['Startup value (mm$)'])
spearman_corr_0_10 = variables_except_last.corrwith(variables_0_10_new['Startup value (mm$)'], method='spearman')


In [ ]:
# Select all columns except the last one
variables_except_last = variables_10_20_new.iloc[:, :-1]

# Calculate the correlation between the selected columns and the 'Startup value (mm$)' column
pearson_corr_10_20 = variables_except_last.corrwith(variables_10_20_new['Startup value (mm$)'])
spearman_corr_10_20 = variables_except_last.corrwith(variables_10_20_new['Startup value (mm$)'], method='spearman')


In [ ]:
# Select all columns except the last one
variables_except_last = variables_20_30_new.iloc[:, :-1]

# Calculate the correlation between the selected columns and the 'Startup value (mm$)' column
pearson_corr_20_30 = variables_except_last.corrwith(variables_20_30_new['Startup value (mm$)'])
spearman_corr_20_30 = variables_except_last.corrwith(variables_20_30_new['Startup value (mm$)'], method='spearman')


In [ ]:
# Select all columns except the last one
variables_except_last = variables_30_new.iloc[:, :-1]

# Calculate the correlation between the selected columns and the 'Startup value (mm$)' column
pearson_corr_30 = variables_except_last.corrwith(variables_30_new['Startup value (mm$)'])
spearman_corr_30 = variables_except_last.corrwith(variables_30_new['Startup value (mm$)'], method='spearman')


In [ ]:
variables_30_new

# 0-10 years

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt



correlation_matrix = variables_0_10_new.corr(method='spearman')
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f",annot_kws={"size": 8})


plt.title('Correlation Matrix Heatmap: career 0 - 10 years (Spearman)')
plt.show()




In [ ]:

correlation_matrix = variables_0_10_new.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f",annot_kws={"size": 8})


plt.title('Correlation Matrix Heatmap: career 0 - 10 years (Pearson)')
plt.show()


In [ ]:

df=variables_0_10_new
x_vars = ['paper frequency','paper citation frequency','paper citation per paper',\
                                                                                'author position','is_corresponding',\
                                                                                'grant number', 'grant money','grant money per grant','grant series_u','grant series_r',\
                                                                                'grant series_t','grant series_p','grant duration per grant' ,  \
                                                                                'patents citations frequency' ,'patents frequency']
correlations = df[x_vars + ['Startup value (mm$)']].corr()['Startup value (mm$)'].drop('Startup value (mm$)')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value (mm$)')
plt.title('Correlation with Startup value (mm$): career 0 - 10 years (Pearson)')
plt.show()


In [ ]:

df=variables_0_10_new
x_vars = ['paper frequency','paper citation frequency','paper citation per paper',\
                                                                                'author position','is_corresponding',\
                                                                                'grant number', 'grant money','grant money per grant','grant series_u','grant series_r',\
                                                                                'grant series_t','grant series_p','grant duration per grant' ,  \
                                                                                'patents citations frequency' ,'patents frequency']
correlations = df[x_vars + ['Startup value (mm$)']].corr(method='spearman')['Startup value (mm$)'].drop('Startup value (mm$)')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value (mm$)')
plt.title('Correlation with Startup value (mm$): career 0 - 10 years (Spearman)')
plt.show()


In [ ]:

df=variables_0_10_new
x_vars = ['paper frequency','paper citation frequency','paper citation per paper',\
                                                                                'author position','is_corresponding',\
                                                                                'grant number', 'grant money','grant money per grant','grant series_u','grant series_r',\
                                                                                'grant series_t','grant series_p','grant duration per grant' ,  \
                                                                                'patents citations frequency' ,'patents frequency']
correlations = df[x_vars + ['Startup value_0/1']].corr()['Startup value_0/1'].drop('Startup value_0/1')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value_0/1')
plt.title('Correlation with Startup value_0/1: career 0 - 10 years (Pearson)')
plt.show()


In [ ]:

df=variables_0_10_new
x_vars = ['paper frequency','paper citation frequency','paper citation per paper',\
                                                                                'author position','is_corresponding',\
                                                                                'grant number', 'grant money','grant money per grant','grant series_u','grant series_r',\
                                                                                'grant series_t','grant series_p','grant duration per grant' ,  \
                                                                                'patents citations frequency' ,'patents frequency']
correlations = df[x_vars + ['Startup value_0/1']].corr(method='spearman')['Startup value_0/1'].drop('Startup value_0/1')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value_0/1')
plt.title('Correlation with Startup value_0/1: career 0 - 10 years (Spearman)')
plt.show()


# 10-20 years

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame containing y and x variables
correlation_matrix = variables_10_20_new.corr(method='spearman')
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f",annot_kws={"size": 8})
plt.title('Correlation Matrix Heatmap: career 10 - 20 years (Spearman)')
plt.show()


In [ ]:
correlation_matrix = variables_10_20_new.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f",annot_kws={"size": 8})
plt.title('Correlation Matrix Heatmap: career 10 - 20 years (Pearson)')
plt.show()

In [ ]:

df=variables_10_20_new
x_vars = ['paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
          'author position','author position_delta','is_corresponding','is_corresponding_delta',\
          'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
          'grant series_t','grant series_p','grant duration per grant' ,  \
          'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta']
correlations = df[x_vars + ['Startup value (mm$)']].corr()['Startup value (mm$)'].drop('Startup value (mm$)')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value (mm$)')
plt.title('Correlation with Startup value (mm$): career 10 - 20 years (Pearson)')
plt.show()


In [ ]:

df=variables_10_20_new
x_vars = ['paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
          'author position','author position_delta','is_corresponding','is_corresponding_delta',\
          'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
          'grant series_t','grant series_p','grant duration per grant' ,  \
          'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta']
correlations = df[x_vars + ['Startup value (mm$)']].corr(method='spearman')['Startup value (mm$)'].drop('Startup value (mm$)')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value (mm$)')
plt.title('Correlation with Startup value (mm$): career 10 - 20 years (Spearman)')
plt.show()


In [ ]:

df=variables_10_20_new
x_vars = ['paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
          'author position','author position_delta','is_corresponding','is_corresponding_delta',\
          'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
          'grant series_t','grant series_p','grant duration per grant' ,  \
          'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta']
correlations = df[x_vars + ['Startup value_0/1']].corr()['Startup value_0/1'].drop('Startup value_0/1')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value_0/1')
plt.title('Correlation with Startup value_0/1: career 10 - 20 years (Pearson)')
plt.show()


In [ ]:

df=variables_10_20_new
x_vars = ['paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
          'author position','author position_delta','is_corresponding','is_corresponding_delta',\
          'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
          'grant series_t','grant series_p','grant duration per grant' ,  \
          'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta']
correlations = df[x_vars + ['Startup value_0/1']].corr(method='spearman')['Startup value_0/1'].drop('Startup value_0/1')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value_0/1')
plt.title('Correlation with Startup value_0/1: career 10 - 20 years (Spearman)')
plt.show()


# 20-30 years

In [ ]:
correlation_matrix = variables_20_30_new.corr(method='spearman')
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f",annot_kws={"size": 8})
plt.title('Correlation Matrix Heatmap: career 20 - 30 years (Spearman)')
plt.show()

In [ ]:
correlation_matrix = variables_20_30_new.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f",annot_kws={"size": 8})
plt.title('Correlation Matrix Heatmap: career 20 - 30 years (Pearson)')
plt.show()

In [ ]:

df=variables_20_30_new
x_vars = ['paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
          'author position','author position_delta','is_corresponding','is_corresponding_delta',\
          'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
          'grant series_t','grant series_p','grant duration per grant' ,  \
          'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta']
correlations = df[x_vars + ['Startup value (mm$)']].corr()['Startup value (mm$)'].drop('Startup value (mm$)')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value (mm$)')
plt.title('Correlation with Startup value (mm$): career 20 - 30 years (Pearson)')
plt.show()


In [ ]:

df=variables_20_30_new
x_vars = ['paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
          'author position','author position_delta','is_corresponding','is_corresponding_delta',\
          'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
          'grant series_t','grant series_p','grant duration per grant' ,  \
          'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta']
correlations = df[x_vars + ['Startup value (mm$)']].corr(method='spearman')['Startup value (mm$)'].drop('Startup value (mm$)')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value (mm$)')
plt.title('Correlation with Startup value (mm$): career 20 - 30 years (Spearman)')
plt.show()


In [ ]:

df=variables_20_30_new
x_vars = ['paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
          'author position','author position_delta','is_corresponding','is_corresponding_delta',\
          'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
          'grant series_t','grant series_p','grant duration per grant' ,  \
          'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta']
correlations = df[x_vars + ['Startup value_0/1']].corr()['Startup value_0/1'].drop('Startup value_0/1')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value_0/1')
plt.title('Correlation with Startup value_0/1: career 20 - 30 years (Pearson)')
plt.show()


In [ ]:

df=variables_20_30_new
x_vars = ['paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
          'author position','author position_delta','is_corresponding','is_corresponding_delta',\
          'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
          'grant series_t','grant series_p','grant duration per grant' ,  \
          'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta']
correlations = df[x_vars + ['Startup value_0/1']].corr(method='spearman')['Startup value_0/1'].drop('Startup value_0/1')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value_0/1')
plt.title('Correlation with Startup value_0/1: career 20 - 30 years (Spearman)')
plt.show()


# > 30 years

In [ ]:
correlation_matrix = variables_30_new.corr(method='spearman')
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f",annot_kws={"size": 8})
plt.title('Correlation Matrix Heatmap: career over 30 years (Spearman)')
plt.show()

In [ ]:
correlation_matrix = variables_30_new.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f",annot_kws={"size": 8})
plt.title('Correlation Matrix Heatmap: career over 30 years (Pearson)')
plt.show()


In [ ]:

df=variables_30_new
x_vars = ['paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
          'author position','author position_delta','is_corresponding','is_corresponding_delta',\
          'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
          'grant series_t','grant series_p','grant duration per grant' ,  \
          'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta']
correlations = df[x_vars + ['Startup value (mm$)']].corr()['Startup value (mm$)'].drop('Startup value (mm$)')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value (mm$)')
plt.title('Correlation with Startup value (mm$): career over 30 years (Pearson)')
plt.show()


In [ ]:

df=variables_30_new
x_vars = ['paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
          'author position','author position_delta','is_corresponding','is_corresponding_delta',\
          'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
          'grant series_t','grant series_p','grant duration per grant' ,  \
          'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta']
correlations = df[x_vars + ['Startup value (mm$)']].corr(method='spearman')['Startup value (mm$)'].drop('Startup value (mm$)')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value (mm$)')
plt.title('Correlation with Startup value (mm$): career over 30 years (Spearman)')
plt.show()


In [ ]:

df=variables_30_new
x_vars = ['paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
          'author position','author position_delta','is_corresponding','is_corresponding_delta',\
          'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
          'grant series_t','grant series_p','grant duration per grant' ,  \
          'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta']
correlations = df[x_vars + ['Startup value_0/1']].corr()['Startup value_0/1'].drop('Startup value_0/1')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value_0/1')
plt.title('Correlation with Startup value_0/1: career over 30 years (Pearson)')
plt.show()


In [ ]:

df=variables_30_new
x_vars = ['paper frequency','paper frequency_delta','paper citation frequency','paper citation frequency_delta','paper citation per paper',\
          'author position','author position_delta','is_corresponding','is_corresponding_delta',\
          'grant number','grant number_delta', 'grant money','grant money_delta','grant money per grant','grant series_u','grant series_r',\
          'grant series_t','grant series_p','grant duration per grant' ,  \
          'patents citations frequency','patents citations frequency_delta' ,'patents frequency','patents frequency_delta']
correlations = df[x_vars + ['Startup value_0/1']].corr(method='spearman')['Startup value_0/1'].drop('Startup value_0/1')
correlations.plot(kind='bar')
plt.xlabel('Variable')
plt.ylabel('Correlation with Startup value_0/1')
plt.title('Correlation with Startup value_0/1: career over 30 years (Spearman)')
plt.show()


In [ ]:
import pandas as pd
import statsmodels.api as sm

df = variables_10_20_new
# Assuming df is your DataFrame
df.fillna(0, inplace=True)


# Define predictor variables (X) and response variable (y)
X = df.iloc[:,0:-2]  # Include all your predictor variables here
y = df.iloc[:,-1]  # Binary response variable (0 or 1)

# Add intercept term to predictor variables
X = sm.add_constant(X)

# Fit probit model
probit_model = sm.Probit(y, X).fit(method='bfgs')

# Print summary of the model
print(probit_model.summary())

# Make predictions (optional)
predictions = probit_model.predict(X)
